## Import required libraries

In [2]:
import requests
import pandas as pd
import numpy as np

## Scraping Wikipedia page to get Toronto Neighbourhoods dataframe

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [8]:
r = requests.get(url) #do I need custom header here?

In [18]:
dfs=pd.read_html(r.text, header=0)

In [22]:
toronto_neis = dfs[0]
toronto_neis.head()

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
toronto_neis.shape

(180, 3)

## Let's clean the dataframe

In [25]:
toronto_neis['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [27]:
toronto_neis = toronto_neis[toronto_neis.Borough != 'Not assigned']
toronto_neis.shape

(103, 3)

In [35]:
toronto_neis.loc[toronto_neis['Neighborhood'] == 'Not Assigned']

,Post Code,Borough,Neighborhood
2,M3A,North York,Not Assigned
3,M4A,North York,Not Assigned
4,M5A,Downtown Toronto,Not Assigned
5,M6A,North York,Not Assigned
6,M7A,Downtown Toronto,Not Assigned
8,M9A,Etobicoke,Not Assigned
9,M1B,Scarborough,Not Assigned
11,M3B,North York,Not Assigned
12,M4B,East York,Not Assigned
13,M5B,Downtown Toronto,Not Assigned
